# scRNA-Seq pre-processing

Pre-processing pipleine with scanpy

In [70]:

import os 
import pandas as pd
import numpy as np
import scanpy as sc
import scipy as sci
import matplotlib.pyplot as plt
import anndata as ad

In [72]:
# Read features
url = 'https://raw.githubusercontent.com/jamesfread/scRNAseq_Preprocessing/main/Data/Donor2_5yr_Unstim_features.tsv'
ft = pd.read_csv(url, sep='\t', header=None)

# Read barcodes
url = "https://raw.githubusercontent.com/jamesfread/scRNAseq_Preprocessing/main/Data/Donor2_5yr_Unstim_barcodes.tsv"
bc = pd.read_csv(url, sep='\t', header=None)

mat = ad.read_mtx('Data/Donor2_5yr_Unstim_matrix.mtx')

Check data

In [ ]:
print(ft.head(5))
print(bc.head(5))
mat

Add barcode and gene names to anndata object

In [ ]:
mat.obs_names = ft[1]
mat.var_names = bc[0]
print(mat.obs_names[:10])
print(mat.var_names[:10])

Continue here:
https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering.html

https://www.kallistobus.tools/tutorials/kb_getting_started/python/kb_intro_2_python/
